## 导出特征向量

In [2]:
import os
import h5py

from keras.models import *
from keras.layers import *
from keras.applications import *
from keras.preprocessing.image import *

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/matplotlib/__init__.py:1067: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #2
  (fname, cnt))
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/matplotlib/__init__.py:1067: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #3
  (fname, cnt))


In [ ]:
# 获得当前目录
current_dir = os.getcwd()
def export_feature(MODEL, image_size, batch_size, preprocess_input=None):
    width = image_size[0]
    height = image_size[1]
    input_tensor = Input((height, width, 3))
    x = input_tensor
    if preprocess_input:
        x = Lambda(preprocess_input)(x)
    base_model = MODEL(input_tensor=x, weights='imagenet', include_top=False)
#     base_model.save_weights(base_model.name+'-imagenet.h5')
    
    model = Model(base_model.input, GlobalAveragePooling2D()(base_model.output))

    gen = ImageDataGenerator()
    
    train_generator = gen.flow_from_directory(current_dir + '/data/train', image_size, shuffle=False, 
                                              batch_size=batch_size)
    test_generator = gen.flow_from_directory(current_dir + "/data/test", image_size, shuffle=False, 
                                             batch_size=batch_size, class_mode=None)

    train_feature = model.predict_generator(train_generator, train_generator.samples, workers=8, verbose=1)
    test_feature = model.predict_generator(test_generator, test_generator.samples, workers=8, verbose=1)
    with h5py.File("feature_%s.h5"%base_model.func_name) as h:
        h.create_dataset("train", data=train_feature)
        h.create_dataset("test", data=test_feature)
        h.create_dataset("label", data=train_generator.classes)

In [6]:
export_feature(ResNet50, (224, 224), batch_size=1)

Found 25000 images belonging to 2 classes.
Found 12500 images belonging to 1 classes.
12500/12500 [==============================] - 348s 28ms/step


AttributeError: 'function' object has no attribute 'func_name'

In [ ]:
export_feature(InceptionV3, (299, 299), batch_size=16, preprocess_input=inception_v3.preprocess_input)

In [ ]:
export_feature(Xception, (299, 299), batch_size=16, preprocess_input=Xception.preprocess_input)

In [ ]:
export_feature(DenseNet201, (224, 224), batch_size=16, preprocess_input=densenet.preprocess_input)

In [ ]:
export_feature(DenseNet169, (224, 224), batch_size=16, preprocess_input=densenet.preprocess_input)